In [38]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Creation Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [39]:
marlowe-cli --version

marlowe-cli 0.0.10.0


In [40]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [41]:
git rev-parse HEAD

c51079be5e83ad6ad059e2c7d439aa4df0499308


### Setup the faucet.

Set the location of keys.

In [42]:
TREASURY=treasury

Set the faucet.

In [43]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq2a4cdym9mn38nf894p5sjqurns4ehjm3pt5t8l4djcnzqjzztcr


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [44]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [45]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [46]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [47]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vpqm0xvgpjnk7v400fr4c48dfl7kd3vtgkl9tvjnn837dtgwxffz7


Fund the address.

In [48]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "7b73927aa3d15f79d7580aa24a9b48a3f83ed656f630726e1e81e778b1b1bb35"


Tip of the blockchain

In [49]:
TIP=$(cardano-cli query tip "${MAGIC[@]}" | jq '.slot')
echo "TIP: $TIP"
# NOW="$((TIP*SLOT_LENGTH+SLOT_OFFSET))"
# HOUR="$((3600*1000))"

TIP: 6363627


Create non-ADA tokens needed to reproduce this bug

In [50]:
MINT_EXPIRES=$((TIP + 1000000))
CUR1_NAME=Pear
marlowe-cli util mint "${MAGIC[@]}" \
  --socket-path "$CARDANO_NODE_SOCKET_PATH" \
  --issuer "$PARTY_ADDR:$PARTY_SKEY" \
  --expires "$MINT_EXPIRES" \
  --out-file /dev/null \
  --submit=600 \
  "$CUR1_NAME:$PARTY_ADDR"
CUR2_NAME=Orange
marlowe-cli util mint "${MAGIC[@]}" \
  --socket-path "$CARDANO_NODE_SOCKET_PATH" \
  --issuer "$PARTY_ADDR:$PARTY_SKEY" \
  --expires "$MINT_EXPIRES" \
  --out-file /dev/null \
  --submit=600 \
  "$CUR2_NAME:$PARTY_ADDR"
CUR3_NAME=Banana
marlowe-cli util mint "${MAGIC[@]}" \
  --socket-path "$CARDANO_NODE_SOCKET_PATH" \
  --issuer "$PARTY_ADDR:$PARTY_SKEY" \
  --expires "$MINT_EXPIRES" \
  --out-file /dev/null \
  --submit=600 \
  "$CUR3_NAME:$PARTY_ADDR"


Fee: Lovelace 173157
Size: 262 / 16384 = 1%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%

Fee: Lovelace 182793
Size: 380 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%

Fee: Lovelace 192341
Size: 496 / 16384 = 3%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%


## The Contract

Set parameters

In [51]:
MINIMUM_ADA=3000000

Create the contract.

In [83]:
echo '"close"' > close-1.contract
cat close-1.contract

"close"


## Run the Contract

### Transaction 1. Create the contract

See what UTxOs the transaction-creation will have available to select from.

In [63]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6d1fc28320de70258753ff5cb3d85daa20789e26dab01c0efbacde0345398ca3     0        246357129 lovelace + TxOutDatumNone
6d1fc28320de70258753ff5cb3d85daa20789e26dab01c0efbacde0345398ca3     1        1025780 lovelace + 1 1f872bc772d6672632e68c2c95730f99ff0278ffd73016e03b9681b1.50656172 + TxOutDatumNone
6d1fc28320de70258753ff5cb3d85daa20789e26dab01c0efbacde0345398ca3     2        1034400 lovelace + 1 1f872bc772d6672632e68c2c95730f99ff0278ffd73016e03b9681b1.4f72616e6765 + TxOutDatumNone
6d1fc28320de70258753ff5cb3d85daa20789e26dab01c0efbacde0345398ca3     3        1034400 lovelace + 1 1f872bc772d6672632e68c2c95730f99ff0278ffd73016e03b9681b1.42616e616e61 + TxOutDatumNone


Build the transaction.

In [64]:
CONTRACT_ID=$(
marlowe create \
  --core-file close-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign close-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 492970f209d4541274b5d7742243b2e956ed86ae77a2fcbf6f15308726ce9c0d#1


Sign the transaction.

In [65]:
cardano-cli transaction sign \
  --tx-body-file close-1.txbody \
  --out-file     close-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [66]:
marlowe submit close-1.tx

"{\"blockHeaderHash\":\"d6584a51fcd062f9f30e5457b7cefd72c1b562e1f00f895a5bafcf4056fa8d82\",\"blockNo\":290681,\"slotNo\":6291297}"


Watch the contract.

In [67]:
marlowe add "$CONTRACT_ID"

492970f209d4541274b5d7742243b2e956ed86ae77a2fcbf6f15308726ce9c0d#1


In [68]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 492970f209d4541274b5d7742243b2e956ed86ae77a2fcbf6f15308726ce9c0d (creation)
ContractId:      492970f209d4541274b5d7742243b2e956ed86ae77a2fcbf6f15308726ce9c0d#1
SlotNo:          6291297
BlockNo:         290681
BlockId:         d6584a51fcd062f9f30e5457b7cefd72c1b562e1f00f895a5bafcf4056fa8d82
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    Close



View the contract's UTxO.

In [69]:
CONTRACT_ADDRESS=$(marlowe-cli contract address)
echo "$CONTRACT_ADDRESS"

addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


In [70]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDRESS" | sed -n -e "1,2p;/${CONTRACT_ID//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
492970f209d4541274b5d7742243b2e956ed86ae77a2fcbf6f15308726ce9c0d     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "b090d3f6a7f54568564b4ee66cdc80306ff8192f9015e54e198565f673f8cb2a"


Advance the contract

In [71]:
marlowe advance --change-address "$PARTY_ADDR" --manual-sign "$PARTY_SKEY" --contract "$CONTRACT_ID"

ApplyFailed (ApplyInputsConstraintsBuildupFailed UnableToDetermineTransactionTimeout)


: 1

## Cleanup

Remove the contract from history tracking.

In [72]:
marlowe rm "$CONTRACT_ID"

af0d5d2fb5b4089968b61176a4f5be72a085e6a9c73f2de5e695deb92df5f55d#1


In [73]:
marlowe ls

Burn the native tokens

In [74]:
marlowe-cli util burn \
  ${MAGIC[@]} \
  --socket-path "$CARDANO_NODE_SOCKET_PATH" \
  --issuer "$PARTY_ADDR:$PARTY_SKEY" \
  --expires $MINT_EXPIRES \
  --token-provider "$PARTY_ADDR:$PARTY_SKEY" \
  --submit 600 \
  --out-file /dev/null


Fee: Lovelace 188513
Size: 308 / 16384 = 1%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
PolicyID "1f872bc772d6672632e68c2c95730f99ff0278ffd73016e03b9681b1"


Consolidate the UTxOs at the addresses.

In [75]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "06aeff30a5c36a46293956b9f31cb079e4624d659dffc51a9383de650975586d"


Send the funds back to the faucet.

In [76]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "1e753fe64ed9390518a20d9eea13e5972ab5257f6042fd4363604d3181dbee65"


See that the funds have been returned to the faucet.

In [78]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
